<a href="https://colab.research.google.com/github/TimkaThymineTimiryazevskaya/sf_data_science/blob/main/%D0%A1%D1%80%D0%B0%D0%B2%D0%BD%D0%B5%D0%BD%D0%B8%D0%B5_%D0%BC%D0%B5%D1%82%D0%BE%D0%B4%D0%BE%D0%B2_%D0%BE%D1%82%D0%B1%D0%BE%D1%80%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [53]:
%%capture
!wget https://www.dropbox.com/s/64ol9q9ssggz6f1/data_ford_price.xlsx



In [54]:
data = pd.read_excel('data_ford_price.xlsx') 
data.head()

,price,year,condition,cylinders,odometer,title_status,transmission,drive,size,lat,long,weather
0,43900,2016,4,6,43500,clean,automatic,4wd,full-size,36.471500,-82.483400,59.0
1,15490,2009,2,8,98131,clean,automatic,4wd,full-size,40.468826,-74.281734,52.0
2,2495,2002,2,8,201803,clean,automatic,4wd,full-size,42.477134,-82.949564,45.0
3,1300,2000,1,8,170305,rebuilt,automatic,4wd,full-size,40.764373,-82.349503,49.0
4,13865,2010,3,8,166062,clean,automatic,4wd,NaN,49.210949,-123.114720,NaN


In [55]:

import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

In [56]:
data = data.drop(columns = ['title_status','transmission','drive', 'size','condition'])

In [57]:
y = data['price']
x = data.drop(columns='price')
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
X_train['weather'] = X_train['weather'].fillna(np.round(np.mean(X_train['weather']),0))
X_test['weather'] = X_test['weather'].fillna(np.round(np.mean(X_train['weather']),0))

**RFE**

In [58]:
from sklearn.feature_selection import RFE
 
estimator = LinearRegression()
selector = RFE(estimator, n_features_to_select=3, step=1)
selector = selector.fit(X_train, y_train)

selector.get_feature_names_out()


array(['year', 'cylinders', 'lat'], dtype=object)

In [64]:
x_train_rfe = X_train[['year', 'cylinders', 'lat']]
x_test_rfe = X_test[['year', 'cylinders', 'lat']]

In [65]:
lr_model = LinearRegression()
lr_model.fit(x_train_rfe, y_train)
y_test_predict = lr_model.predict(x_test_rfe)
print("Test R^2: {:.3f}".format(r2_score(y_test, y_test_predict)))

Test R^2: 0.386


**Kbest**

In [66]:
from sklearn.feature_selection import SelectKBest, f_regression
selector = SelectKBest(f_regression, k=3)
selector.fit(X_train, y_train)
 
selector.get_feature_names_out()

array(['year', 'cylinders', 'odometer'], dtype=object)

In [68]:
x_train_kbest = X_train[['year', 'cylinders', 'odometer']]
x_test_kbest = X_test[['year', 'cylinders', 'odometer']]

In [69]:
lr_model = LinearRegression()
lr_model.fit(x_train_kbest, y_train)
y_test_predict = lr_model.predict(x_test_kbest)
print("Test R^2: {:.3f}".format(r2_score(y_test, y_test_predict)))

Test R^2: 0.413


Вывод: исходя из метрики R^2 метод отбора KBest на основе фильтров показал результат лучше RFE